In [44]:
import npc_lims
from ccfwidget import CCFWidget
import dataframes
import ipywidgets as widgets
import numpy as np

In [115]:
electrodes = dataframes.get_dfs()['electrodes']
electrodes_dr = electrodes[electrodes['is_dynamic_routing'] == True]
electrodes_templeton = electrodes[electrodes['is_templeton'] == True]
IS_DYNAMIC_ROUTING = True

In [116]:
electrodes_dr = electrodes_dr.dropna()
electrodes_templeton = electrodes_templeton.dropna()

if IS_DYNAMIC_ROUTING:
    electrodes_structures = electrodes_dr.copy()
else:
    electrodes_structures = electrodes_templeton[~(electrodes_dr['structure'].str.islower())]

In [117]:
electrodes_structures_dict = {}

In [118]:
for session in electrodes_structures['session_id'].unique():
    electrodes_dr_session = electrodes_structures[electrodes_structures['session_id'] == session]

    for group in electrodes_dr_session['group_name'].unique():
        electrodes_dr_session_group = electrodes_dr_session[electrodes_dr_session['group_name'] == group]
        xyz_coords = electrodes_dr_session_group[['x', 'y', 'z']].to_numpy()

        for area in electrodes_dr_session_group['structure'].unique():
            if area in electrodes_structures_dict:
                electrodes_structures_dict[area].append(xyz_coords)
            else:
                electrodes_structures_dict[area] = [xyz_coords]

In [119]:
electrodes_structures_dict['PG']

[array([[9150., 6950., 6675.],
        [9150., 6950., 6675.],
        [9150., 6950., 6675.],
        ...,
        [9050., 3625., 3900.],
        [9025., 3600., 3900.],
        [9025., 3600., 3900.]])]

In [120]:
area_dropdown = widgets.Dropdown(
    options=sorted(list(electrodes_structures_dict.keys())),
    value=sorted(list(electrodes_structures_dict.keys()))[0],
    description='CCF Area:',
    disabled=False,
)
area_dropdown

Dropdown(description='CCF Area:', options=('ACAd', 'ACAv', 'ACB', 'AD', 'AId', 'AIv', 'AON', 'APN', 'AQ', 'AUD…

In [121]:
ccf = CCFWidget(markers=electrodes_structures_dict[area_dropdown.value], selected_acronyms=[area_dropdown.value])
ccf

CCFWidget(children=(VBox(children=(Viewer(background=(0.85, 0.85, 0.85), camera=array([[ 1.3441567e+03, -2.172…